In [4]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time
import datetime as dt

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','size']

In [18]:
start = dt.datetime.strftime(dt.datetime.now()-dt.timedelta(365*2),format='%Y%m%d')
start = int(start)
print(start)
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
print(df_d.columns)
print(df_d.shape)

20180409
select * from stock_day where date>=20180409
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0'],
      dtype='object')
(700929, 10)


In [19]:
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt'] /= 1e4
df = df_d

periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [21]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
print(len(index))

174


In [22]:
df.loc[index,['open','close','amt','pct','adj_factor','close/30ma','close/60ma','close/120ma']].sort_values('date',ascending=False)[:50]

open      close         amt        pct  adj_factor  \
code      date                                                                  
002506.SZ 2020-04-08   15.54585   15.39294   60.191042  -1.628664       5.097   
          2020-04-07   15.49488   15.64779   90.621543   3.020134       5.097   
          2020-04-03   15.49488   15.18906   79.609115  -2.931596       5.097   
          2020-04-02   15.54585   15.64779   95.018972  -0.647249       5.097   
002411.SZ 2020-04-02   32.16152   33.07804   33.476047   2.187902       4.166   
002506.SZ 2020-04-01   15.39294   15.74973  107.491603   1.980198       5.097   
002411.SZ 2020-04-01   32.32816   32.36982   38.996958   0.647668       4.166   
300618.SZ 2020-03-31  106.54761  105.06366   31.511127   0.261438       2.283   
002411.SZ 2020-03-31   34.45282   32.16152   51.232793  -5.738706       4.166   
002506.SZ 2020-03-31   16.41234   15.44391  148.703239  -7.621951       5.097   
002353.SZ 2020-03-31  178.39740  180.10806   57.088282   2.551020       8.146   
          2020-03-30  178.07156  175.62776   62.894394  -5.397104       8.146   
300618.SZ 2020-03-30  106.09101  104.78970   32.823085  -3.123681       2.283   
002411.SZ 2020-03-30   34.57780   34.11954   42.732068  -2.500000       4.166   
002506.SZ 2020-03-30   16.97301   16.71816  111.867122  -1.501502       5.097   
002353.SZ 2020-03-27  192.73436  185.64734   48.624828  -2.104811       8.146   
002411.SZ 2020-03-27   33.74460   34.99440   66.133869  -4.977376       4.166   
002506.SZ 2020-03-27   17.83950   16.97301  241.597746 -10.000000       5.097   
002411.SZ 2020-03-26   38.28554   36.82744   62.508780  -9.979633       4.166   
300618.SZ 2020-03-23  111.86700  105.88554   43.920891  -8.520710       2.283   
000662.SZ 2020-02-04   20.18362   20.90878   18.364174  -6.738544       6.043   
600338.SH 2019-08-12   21.85267   21.96313   12.747738   0.845309       1.841   
          2019-08-09   22.40497   21.77903   14.539615  -2.069536       1.841   
          2019-08-08   22.69953   22.23928   15.319773  -1.628664       1.841   
          2019-08-07   22.09200   22.60748   33.022005   3.628692       1.841   
          2019-08-06   22.11041   21.81585   29.034593  -5.048077       1.841   
002341.SZ 2019-06-27   46.32320   46.32320    3.328808  -9.967846       8.272   
300090.SZ 2019-06-11   10.05054   10.26903    4.549120  10.156250       7.283   
          2019-06-10    8.88526    9.32224    5.356801  10.344828       7.283   
600179.SH 2019-06-10    8.99878    9.38315   11.962877   2.977667       2.261   
002247.SZ 2019-06-10   18.81230   17.19055    8.448482  -9.863946       6.487   
300156.SZ 2019-06-10   22.33656   21.60120    6.853140  -6.000000       9.192   
300178.SZ 2019-06-10   26.82140   26.27588    8.813958  -3.344482       4.546   
600179.SH 2019-06-06    9.56403    9.11183   12.599530  -8.409091       2.261   
002537.SZ 2019-06-06   48.76207   46.57359   13.709280  -4.888268       6.839   
600179.SH 2019-06-05   10.73975    9.94840   16.660116  -9.836066       2.261   
300090.SZ 2019-06-05    8.81243    9.32224   17.647965  -4.477612       7.283   
002247.SZ 2019-06-05   18.74743   18.35821    3.505117  -1.393728       6.487   
300090.SZ 2019-06-04    9.75922    9.75922    0.492195 -10.067114       7.283   
002247.SZ 2019-06-04   18.81230   18.61769    3.376082  -0.692042       6.487   
300090.SZ 2019-06-03   10.85167   10.85167    0.779538  -9.696970       7.283   
002247.SZ 2019-06-03   19.52587   18.74743    5.134943  -3.344482       6.487   
600179.SH 2019-05-31   11.03368   10.46843   13.325823  -5.122951       2.261   
002470.SZ 2019-05-29   17.54559   17.71635    8.120078   0.000000       4.269   
002247.SZ 2019-05-28   18.87717   19.13665   13.281964  -2.317881       6.487   
002470.SZ 2019-05-28   17.84442   17.71635   11.056820  -1.190476       4.269   
002247.SZ 2019-05-27   19.59074   19.59074    1.579339  -9.850746       6.487   
002470.SZ 2019-05-27   17.41752   17.92980   14.787409   2